In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [3]:
from nflows.flows import MaskedAutoregressiveFlow

from counterfactuals.datasets.heloc import HelocDataset
from counterfactuals.datasets.moons import MoonsDataset
from counterfactuals.datasets.law import LawDataset
from counterfactuals.datasets.compas import CompasDataset

from counterfactuals.optimizers.approach_three import ApproachThree

from counterfactuals.metrics.metrics import (
    perc_valid_cf,
    perc_valid_actionable_cf,
    continuous_distance,
    categorical_distance,
    distance_l2_jaccard,
    distance_mad_hamming,
    plausibility
)

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

/Users/ofurman/Study/counterfactuals/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset = CompasDataset(file_path="../data/origin/compas_two_years.csv")
train_dataloader = dataset.train_dataloader(batch_size=64, shuffle=True)
test_dataloader = dataset.test_dataloader(batch_size=64, shuffle=False)

In [5]:
flow = MaskedAutoregressiveFlow(features=dataset.X_train.shape[1], hidden_features=4, context_features=1)
cf = ApproachThree(model=flow)

In [6]:
cf.train_model(
    train_loader=train_dataloader,
    test_loader=test_dataloader,
    epochs=50,
    verbose=True
)

Epochs:   4%|▍         | 2/50 [00:00<00:05,  8.48it/s]

Epoch 0, Train: 43.986488342285156, test: 39.80231366838728


Epochs:  24%|██▍       | 12/50 [00:01<00:04,  9.45it/s]

Epoch 10, Train: -0.6626100929514054, test: -1.0727684944868088


Epochs:  44%|████▍     | 22/50 [00:02<00:02,  9.39it/s]

...

Epochs:  64%|██████▍   | 32/50 [00:03<00:01,  9.49it/s]

Epoch 30, Train: -3.480473804858423, test: -3.794318199157715


Epochs:  84%|████████▍ | 42/50 [00:04<00:00,  9.53it/s]

Epoch 40, Train: -4.977698210747011, test: -5.348496266773769


Epochs: 100%|██████████| 50/50 [00:05<00:00,  9.38it/s]


In [7]:
cf.test_model(test_loader=test_dataloader)

              precision    recall  f1-score   support

         0.0       0.75      0.77      0.76       423
         1.0       0.76      0.74      0.75       419

    accuracy                           0.76       842
   macro avg       0.76      0.76      0.76       842
weighted avg       0.76      0.76      0.76       842



In [10]:
Xs_cfs = cf.generate_counterfactuals(Xs=dataset.X_test[:100], ys=dataset.y_test[:100], num_epochs=200, lr=0.005, alpha=20, beta=0.01)
Xs_cfs = torch.concat(Xs_cfs).detach()

0it [00:00, ?it/s]

100it [01:32,  1.08it/s]


In [11]:
X = dataset.X_test[:100]
ys_cfs_pred = cf.predict_model(Xs_cfs)
ys_orig_pred = cf.predict_model(dataset.X_test[:100])
ys_orig = dataset.y_test[:100].flatten()

{
    "valid_cf": perc_valid_cf(ys_orig_pred, y_cf=ys_cfs_pred),
    "valid_cf_for_orig_data": perc_valid_cf(ys_orig, y_cf=ys_cfs_pred),
    # "perc_valid_actionable_cf": perc_valid_actionable_cf(X=dataset.X_test[:100], X_cf=Xs_cfs, y=ys_orig_pred, y_cf=ys_cfs_pred, actionable_features=[1,2]),
    "continuous_distance": continuous_distance(X=X, X_cf=Xs_cfs, continuous_features=dataset.continuous_columns, metric='mad', X_all=dataset.X_test),
    "categorical_distance": categorical_distance(X=X, X_cf=Xs_cfs, categorical_features=dataset.categorical_columns, metric='jaccard', agg='mean'),
    "distance_l2_jaccard": distance_l2_jaccard(X=X, X_cf=Xs_cfs, continuous_features=dataset.continuous_columns, categorical_features=dataset.categorical_columns),
    "distance_mad_hamming": distance_mad_hamming(X=X, X_cf=Xs_cfs,
                                                 continuous_features=dataset.continuous_columns, categorical_features=dataset.categorical_columns, X_all=X, agg='mean'),
    "plausibility": plausibility(
        X, Xs_cfs, ys_orig,
        continuous_features_all=dataset.continuous_columns,
        categorical_features_all=dataset.categorical_columns,
        X_train=dataset.X_train,
        ratio_cont=None
    ),
    "log_density": np.mean(cf.predict_model(test_dataloader))
}

{'valid_cf': 0.77,
 'valid_cf_for_orig_data': 1.0,
 'continuous_distance': 44.1639847685898,
 'categorical_distance': 0.77,
 'distance_l2_jaccard': 0.5472639875705996,
 'distance_mad_hamming': 20.96497410749426,
 'plausibility': 20.804536163113323,
 'log_density': 0.48812351543942994}